### 6 февраля, случайный лес, выборки с новыми фичами

In [47]:
path_to_files = './data/redir'
path_to_data = './data/clear_data'

In [48]:
import pandas as pd
import numpy as np
from utils.feature_generation import get_data_with_feature

In [49]:
# train data is data1 by Denis
train_data = pd.read_csv(path_to_data + '/train_data.csv')

In [50]:
train_data.head(3)

,query,category_id,category_name,is_redirect
0,лаковый тинт для губ,3328459,Jo Malone London,0
1,L:A BRUKET,3328292,L:A BRUKET,1
2,выпрямляющий кондиционер,3327985,Sergey Naumov,0


In [51]:
# TODO пофиксить проблему с косинусным расстоянием
train_data = get_data_with_feature(train_data, path_to_files)

/home/kostya/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [64]:
validate_data.head()

,is_redirect,len_of_query,num_of_word_in_query,len_of_category,num_of_word_in_category,how_match_brands_name_in_query,how_match_products_name_in_query,mean_word_len_in_category,mean_word_len_in_query,max_word_len_in_category,max_word_len_in_query,min_word_len_in_category,min_word_len_in_query,is_query_long,lev_dist,cosine_dist
0,1,8,1,16,3,0,0,4.666667,8.0,8,8,1,8,0,8,0.185108
1,1,4,1,4,1,0,0,4.000000,4.0,4,4,4,4,0,0,0.000000
2,1,3,1,4,1,0,0,4.000000,3.0,4,3,4,3,0,1,0.529906
3,1,10,2,24,3,0,1,7.333333,4.5,13,6,3,3,0,14,0.395354
4,1,7,1,18,2,0,0,8.500000,7.0,10,7,7,7,0,11,0.274179


In [65]:
validate_data.to_csv('validate_with_features.csv', index=False)

In [62]:
train_data.tocsv()

,is_redirect,len_of_query,num_of_word_in_query,len_of_category,num_of_word_in_category,how_match_brands_name_in_query,how_match_products_name_in_query,mean_word_len_in_category,mean_word_len_in_query,max_word_len_in_category,max_word_len_in_query,min_word_len_in_category,min_word_len_in_query,is_query_long,lev_dist,cosine_dist
0,0,20,4,16,3,0,1,4.666667,4.250000,6,7,2,3,0,19,1.0
1,1,10,3,10,3,0,0,2.666667,2.666667,6,6,1,1,0,0,0.0
2,0,24,2,13,2,0,0,6.000000,11.500000,6,12,6,11,0,23,1.0
3,0,23,4,2,1,0,1,2.000000,5.000000,2,9,2,1,0,23,1.0
4,0,11,2,6,1,0,0,6.000000,5.000000,6,7,6,3,0,11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3407,1,19,2,19,2,2,2,9.000000,9.000000,9,9,9,9,0,0,0.0
3408,0,8,1,3,1,1,1,3.000000,8.000000,3,8,3,8,0,7,1.0
3409,0,68,11,5,1,1,7,5.000000,5.272727,5,10,5,3,1,65,1.0
3410,1,5,1,5,1,1,0,5.000000,5.000000,5,5,5,5,0,0,0.0


In [53]:
train_data['cosine_dist'].isna().sum()

2

In [54]:
train_data['cosine_dist'] = train_data['cosine_dist'].fillna(train_data['cosine_dist'].mean())

In [55]:
train_data['cosine_dist'].isna().sum()

0

In [56]:
X = train_data.drop(columns='is_redirect')
y = train_data['is_redirect']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    stratify=y,
    random_state=0,
    shuffle=True,
    test_size=0.2
)

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline


def make_features_pipeline(n_jobs):
    pipeline = Pipeline([
        ('clf', RandomForestClassifier(n_jobs=n_jobs, random_state=1))
    ])
    return pipeline

from pipeline.training import SCORING, STRONG_CV
from pipeline.training import fit_pipeline, cross_validate_pipeline


N_JOBS = 1
N_JOBS_CV = 4

pipeline = make_features_pipeline(N_JOBS)

result = cross_validate_pipeline(
    pipeline,
    X_train,
    y_train,
    cv=STRONG_CV,
    scoring=SCORING,
    n_jobs=N_JOBS_CV,
    verbose=1,
)

pipeline.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, confusion_matrix


print("RF from pipeline:")

print("\nMetrics for train:")
for metric in [roc_auc_score, f1_score, accuracy_score]:
    print(metric.__name__, round(metric(y_train, pipeline.predict(X_train)), 4))

print("\nMetrics for test:")
for metric in [roc_auc_score, f1_score, accuracy_score]:
    print(metric.__name__, round(metric(y_test, pipeline.predict(X_test)), 4))
    
print('Confusion matrix for train:')
print(confusion_matrix(y_train, pipeline.predict(X_train)))

print('Confusion matrix for test:')
print(confusion_matrix(y_test, pipeline.predict(X_test)))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    7.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    7.9s finished


RF from pipeline:

Metrics for train:
roc_auc_score 0.998
f1_score 0.9978
accuracy_score 0.9982

Metrics for test:
roc_auc_score 0.9686
f1_score 0.963
accuracy_score 0.9693
Confusion matrix for train:
[[1598    2]
 [   3 1126]]
Confusion matrix for test:
[[389  11]
 [ 10 273]]


### Валидация

In [57]:
validate_data = pd.read_csv(path_to_data + '/validate_data.csv')

In [58]:
validate_data.head(3)

,query,category_name,category_id,is_redirect
0,колготки,колготки и носки,3245756,1
1,бады,БАДы,3390215,1
2,бад,БАДы,3390215,1


In [59]:
validate_data = get_data_with_feature(validate_data, path_to_files)

In [60]:
validate_data.head(3)

,is_redirect,len_of_query,num_of_word_in_query,len_of_category,num_of_word_in_category,how_match_brands_name_in_query,how_match_products_name_in_query,mean_word_len_in_category,mean_word_len_in_query,max_word_len_in_category,max_word_len_in_query,min_word_len_in_category,min_word_len_in_query,is_query_long,lev_dist,cosine_dist
0,1,8,1,16,3,0,0,4.666667,8.0,8,8,1,8,0,8,0.185108
1,1,4,1,4,1,0,0,4.000000,4.0,4,4,4,4,0,0,0.000000
2,1,3,1,4,1,0,0,4.000000,3.0,4,3,4,3,0,1,0.529906


In [61]:
X_validate = validate_data.drop(columns=['is_redirect'])
y_validate = validate_data['is_redirect']

print("\nMetrics for validate:")
for metric in [roc_auc_score, f1_score, accuracy_score]:
    print(metric.__name__, round(metric(y_validate, pipeline.predict(X_validate)), 4))
    
print('Confusion matrix for validate:')
print(confusion_matrix(y_validate, pipeline.predict(X_validate)))


Metrics for validate:
roc_auc_score 0.5679
f1_score 0.6134
accuracy_score 0.4873
Confusion matrix for validate:
[[ 19 121]
 [  0  96]]
